In [1]:
import random
import multiprocessing
import pandas as pd
from deap import base
from deap import creator
from deap import tools
from datetime import datetime, timedelta

In [2]:
# Load files
path = '../Data/'
festival = 'IFEMA_Festival_Videojuegos_1/'
preferences = 'Preferencias/'
file_name= 'IFEMA_Festival_Videojuegos_v_5.csv'
EVENTO = pd.read_csv(path+festival+file_name)
file_name='PREFERENCIAS_US1.csv'
PREFERENCIAS = pd.read_csv(path+festival+preferences+file_name)
PREFERENCIAS = PREFERENCIAS.sort_values('Id_Recurso')
PREFERENCIAS.head(15)

,Id_Recurso,Preferencia
101,0,0.233295
88,1,0.358799
111,2,0.120212
102,3,0.227418
113,4,0.102221
107,5,0.184035
27,6,0.494820
114,7,0.044136
50,8,0.425795
89,9,0.358799


In [3]:
# Size of the individual
SIZE_INDIVIDUAL = max(EVENTO['Código'])

# Number of generations
NUM_GENERATIONS = 20

# Optimal value to obtain (not reached)
MAX_VALUE = SIZE_INDIVIDUAL + 10 * SIZE_INDIVIDUAL

# Poblation members
SIZE_POBLATION = 300

# CXPB  is the probability with which two individuals are crossed
CXPB = 0.5

# MUTPB is the probability for mutating an individual
MUTPB = 0.2

In [4]:
# Define the fitness function for maximizing the result
creator.create("FitnessMax", base.Fitness, weights=(1.0,))

# Creation of the individual with the fitness function
creator.create("Individual", list, fitness=creator.FitnessMax)

In [5]:
toolbox = base.Toolbox()

# Attribute generator having only 0 or 1 in the body of the individual
toolbox.register("attr_bool", random.randint, 0, 0)

# Structure initializers
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, SIZE_INDIVIDUAL)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

In [6]:
# Enable multiprocessing
pool = multiprocessing.Pool()
toolbox.register("map", pool.map)

In [7]:
# Fitness function
def evalOneMax(individual):
    summa = 0
    if not find_overlapping(individual):
        summa = sum(individual)
        for i in range(0, len(individual)):
            if individual[i] == 1:
                summa = summa + PREFERENCIAS[PREFERENCIAS['Id_Recurso'] == i].Preferencia.values[0]
    return summa,

In [8]:
# Overlapping function to detect time problems in resources
def find_overlapping(individual):
    found = False
    i = 0
    while i < len(individual) and not found:
        # If the resource is selected for the algorithm.
        if individual[i] == 1:
            j = 0
            while j < len(individual) and not found:
                if i != j and individual[j] == 1:
                    # Get date for the first resource.
                    date_1 = datetime.strptime(EVENTO[EVENTO['Código'] == i].Fecha_1.values[0], '%d/%m/%Y')
                    
                    # Get date for the second resource.
                    date_2 = datetime.strptime(EVENTO[EVENTO['Código'] == j].Fecha_1.values[0], '%d/%m/%Y')
                    
                    # Evaluate if the resource is in the same day.
                    if (date_1 - date_2).days == 0:
                        # Get open time for the first resource.
                        ha1 = datetime.strptime(EVENTO[EVENTO['Código'] == i].Hora_apertura_1.values[0], '%H:%M').time()
                        
                        # Get close time for the first resource.
                        hc1 = datetime.strptime(EVENTO[EVENTO['Código'] == i].Hora_cierre_1.values[0], '%H:%M').time()
                        
                        # Get open time for the second resource.
                        ha2 = datetime.strptime(EVENTO[EVENTO['Código'] == j].Hora_apertura_1.values[0], '%H:%M').time()
                        
                        # Get close time for the second resource.
                        hc2 = datetime.strptime(EVENTO[EVENTO['Código'] == j].Hora_cierre_1.values[0], '%H:%M').time()
                        diff_hour = (hc2.hour - ha2.hour)
                        if diff_hour < 6:
                            if (ha1 == ha2) or (ha2 < ha1 < hc2) or (ha1 < ha2 < hc1) or (ha2 < ha1 < hc1 < hc2) \
                            or (ha1 < ha2 < hc2 < hc1):
                                found = True
                j = j + 1
        i = i + 1

    return found

In [9]:
best = [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 
if find_overlapping(best):
    print("Overlapping")
else:
    print("No overlapping")
    
evalOneMax(best)

Overlapping


(0,)

In [10]:
# Record the allowed operations
toolbox.register("evaluate", evalOneMax)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
#toolbox.decorate("evaluate", tools.DeltaPenalty(feasible, 0.0))

In [11]:
# Initial population
pop = toolbox.population(n=SIZE_POBLATION)

In [12]:
# Evaluate the entire population
fitnesses = list(map(toolbox.evaluate, pop))

In [13]:
# Store the values of the fitness function for each individual
for ind, fit in zip(pop, fitnesses):
    ind.fitness.values = fit

In [14]:
# Function for calculating the stats each generation
def calculate_statistics(fits, g=0):
    print("-- Generation %i --" % g)   
    length = len(pop)
    mean = sum(fits) / length
    sum2 = sum(x*x for x in fits)
    std = abs(sum2 / length - mean**2)**0.5
    print("  Min %s" % min(fits))
    print("  Max %s" % max(fits))
    print("  Avg %s" % mean)
    print("  Std %s" % std)

In [15]:
# Gather all the fitnesses in one list and print the stats
fits = [ind.fitness.values[0] for ind in pop]
calculate_statistics(fits)

-- Generation 0 --
  Min 0.0
  Max 0.0
  Avg 0.0
  Std 0.0


In [16]:
# Variable keeping track of the number of generations
g = 0

# Begin the evolution
while max(fits) < MAX_VALUE and g < NUM_GENERATIONS:
    # A new generation
    g = g + 1
    
    # Select the next generation individuals
    offspring = toolbox.select(pop, len(pop))
    
    # Clone the selected individuals
    offspring = list(map(toolbox.clone, offspring))
    
    # Apply crossover and mutation on the offspring
    for child1, child2 in zip(offspring[::2], offspring[1::2]):
        if random.random() < CXPB:
            toolbox.mate(child1, child2)
            del child1.fitness.values
            del child2.fitness.values

    for mutant in offspring:
        if random.random() < MUTPB:
            toolbox.mutate(mutant)
            del mutant.fitness.values
            
    # Evaluate the individuals with an invalid fitness
    invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
    fitnesses = map(toolbox.evaluate, invalid_ind)
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit
        
    pop[:] = offspring
    # Gather all the fitnesses in one list and print the stats
    fits = [ind.fitness.values[0] for ind in pop]
    calculate_statistics(fits, g)

-- Generation 1 --
  Min 0.0
  Max 4.103440204710261
  Avg 0.045366594400922826
  Std 0.39516946248351753
-- Generation 2 --
  Min 0.0
  Max 4.348973402373577
  Avg 0.1790011737540038
  Std 0.6895392330993027
-- Generation 3 --
  Min 0.0
  Max 4.419386516085871
  Avg 0.4524389315580987
  Std 1.0483292455359898
-- Generation 4 --
  Min 0.0
  Max 5.891983665139431
  Avg 0.9015110982194426
  Std 1.4207733423797468
-- Generation 5 --
  Min 0.0
  Max 7.105682295716092
  Avg 1.6036855130036458
  Std 1.6967646431130075
-- Generation 6 --
  Min 0.0
  Max 7.105682295716092
  Avg 2.284382590078719
  Std 1.8452346846900352
-- Generation 7 --
  Min 0.0
  Max 8.01637359978315
  Avg 2.7086528362215216
  Std 2.0638841920002573
-- Generation 8 --
  Min 0.0
  Max 8.657803260251173
  Avg 3.2662326969279563
  Std 2.263962494014596
-- Generation 9 --
  Min 0.0
  Max 8.657803260251173
  Avg 3.408164701571097
  Std 2.656210388188243
-- Generation 10 --
  Min 0.0
  Max 8.657803260251173
  Avg 3.8885662444320

In [17]:
# Find and print best individual:
best_index = fits.index(max(fits))
best_individual = pop[best_index]
print(len(best_individual))
print("Best Individual = ", best_individual, "\n")

114
Best Individual =  [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0] 



In [18]:
for i in range(0, len(best_individual)):
    if best_individual[i] == 1:
        print(i, EVENTO[EVENTO['Código'] == i].Fecha_1.values[0], EVENTO[EVENTO['Código'] == i].Hora_apertura_1.values[0], EVENTO[EVENTO['Código'] == i].Hora_cierre_1.values[0], EVENTO[EVENTO['Código'] == i].Nombre.values[0])

8 16/12/2022 0:00 0:00 Open Mario Kart 8 Deluxe
73 16/12/2022 10:00 22:00 Perritos calientes
74 16/12/2022 10:00 22:00 Hamburguesa
79 16/12/2022 10:00 22:00 Food Truck
82 16/12/2022 10:00 22:00 grido
84 16/12/2022 12:00 23:00 Dominos pizza
85 16/12/2022 12:00 23:00 McDonal's
94 16/12/2022 10:00 21:00 Amazon University Esports
95 16/12/2022 10:00 21:00 Junior Esports
111 16/12/2022 10:00 21:00 Mad Lions
112 16/12/2022 10:00 21:00 Koi
